In [1]:
import requests
import json
import bcrypt
import time

In [ ]:
# Authentication Sorare API with Postman

salt = b"xxx"
password = b"xxx"
hashed_password = bcrypt.hashpw(password, salt)

"""
***Query to https://api.sorare.com/graphql****

mutation SignInMutation($input: signInInput!) {
  signIn(input: $input) {
    currentUser {
      slug
      jwtToken(aud: "databases-project") {
        token
        expiredAt
      }
    }
    errors {
      message
    }
  }
}


***GraphQL Variables****
{
  "input": {
    "email": "xxx",
    "password": "hashed_password"
  }
}

***Response****
{
    "jwt": "token"
    "expiredAt": "2022-12-04T10:36:32Z"
}

"""

In [ ]:
# sorare data from https://api.sorare.com/graphq (access via VPN, CH not allowed)
# Playground: https://api.sorare.com/graphql/playground

url = 'https://api.sorare.com/graphql'
headers = {
  'Content-Type': 'application/json',
  'Authorization': 'Bearer ourJWT', # use jwt
  'JWT-AUD': 'databases-project'
}

In [ ]:
# 1. Fetch all football nfts with pagination (Result: see "./mock-data/raw_all_nfts.json")

# create GraphQL query helper to fetch all nfts
def createAllNftsQuery(endCursor):
    qStart = "{tokens{allNfts(last: 50, sport: FOOTBALL, after: "
    if endCursor == "null":
        qMiddle = "null"
    else:
        qMiddle = f'"{endCursor}"'
    qEnd = """) {
          nodes {
            assetId
            id
            slug
            owner {
              address
              from
            }
            ownershipHistory {
              address
              blockchain
              from
              priceFiat {
                eur
              }
              priceWei
              transferType
            }
          }
          pageInfo {
            endCursor
            hasNextPage
          }
        }
      }
    }"""
    
    return qStart + qMiddle + qEnd



hasNextPage = True # first result has a next page
endCursor = "null" # first endCuror must be null (in string format)
namingCount = 1 # to create seperate .json files for every 50 entries  

# continue until last page
while hasNextPage:
    time.sleep(3) # wait at least 3 seconds, API restriction (20x per Minute)
    query = createAllNftsQuery(endCursor) # with endCursor
    
    # request
    response = requests.post(url, headers=headers, json={'query': query})
    jsonData = response.json()
    endCursor = jsonData['data']['tokens']['allNfts']['pageInfo']['endCursor'] # set new endCursor
    hasNextPage = jsonData['data']['tokens']['allNfts']['pageInfo']['hasNextPage'] # set new hasNextPage
    nfts = jsonData['data']['tokens']['allNfts']['nodes'] # actual nft data

    nfts_data = []
    for nft in nfts:
        nfts_data.append(nft)
    
    # create single nft file with 50 entries (restriction entries per page)
    with open(f"../data/sorare_nfts/nfts_{namingCount}.json", 'w', encoding='utf-8') as f:
        json.dump(nfts_data, f, ensure_ascii=False, indent=4)
    
    namingCount += 1
    

# 1.2 Merge all nfts into one single file
files = [f"../data/sorare_nfts/nfts_{i}.json" for i in range(1, 27620)] # 27619 files; nfts_1 ... nfts_27619

# credits: https://stackoverflow.com/questions/57422734/how-to-merge-multiple-json-files-into-one-file-in-python; Akaisteph7
def mergeJsonFiles(files, filename):
    result = list()
    for f1 in files:
        with open(f1, 'r') as infile:
            result.extend(json.load(infile))

    with open(filename, 'w') as output_file:
        json.dump(result, output_file)

mergeJsonFiles(files, '../data/sorare_all_nfts.json') # merge all files into one single file

In [ ]:
# 2. Fetch all cards according to nfts fetched previously (Result: see "./mock-data/raw_all_cards.json")

# create GraphQL query helper to fetch all cards
def createAllCardsQuery(slugs):
    qStart = "{allCards(slugs: ["
    # insert all slugs
    qMiddle = ""
    for i in slugs:
        qMiddle += f'"{i}",'
    
    qEnd = """]
      ) {
        nodes {
          assetId
          birthTxHash
          blockchainId
          name
          owner {
            address
            from
          }
          ownerSince
          player {
            age
            birthDate
            bestFoot
            firstName
            lastName
            displayName
            height
            weight
            slug
            shirtNumber
            activeClub {
              name
              slug
            }
            lastClub {
              name
              slug
            }
          }
          rarity
          season {
            id
            name
            startYear
          }
        }
      }
    }
    """
    return qStart + qMiddle + qEnd

namingCount = 1

# iterate over all single nft files (50 entries per file)
for i in range(namingCount, 27620):
    time.sleep(3) # wait at least 3 seconds, API restriction (20x per Minute)
    
    with open(f'../data/sorare_nfts/nfts_{i}.json') as f: # open single nft file and load data (50 entries)
        nfts = json.load(f)
    
    # create cards query with all the slugs per file
    slugs = [nft["slug"] for nft in nfts]
    query = createAllCardsQuery(slugs)
    
    # request für cards
    response = requests.post(url, headers=headers, json={'query': query})
    jsonData = response.json()
    cards = jsonData['data']['allCards']['nodes'] # actual cards data
    
    cards_data = []
    for card in cards:
        cards_data.append(card)
        
    # create single card file with 50 entries 
    with open(f"../data/sorare_cards/cards_{namingCount}.json", 'w', encoding='utf-8') as f:
        json.dump(cards_data, f, ensure_ascii=False, indent=4)
    
    namingCount += 1

    
# 2.2 Merge all cards into one single file
files = [f"../data/sorare_cards/cards_{i}.json" for i in range(1, tbd)] # tbd files; cards_1 ... cards_tbd
mergeJsonFiles(files, '../data/sorare_all_cards.json') # merge all files into one single file